In [130]:
using Base.Iterators
using Base.Test

http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
を参考にWord2VecについてJuliaを用いて理解する

In [28]:
text = "The quick brown fox jumps over the lazy dog."

"The quick brown fox jumps over the lazy dog."

In [34]:
remove_period = (text) -> replace(text, ".", "")
split_space = (text) -> split(text, " ")
splitted_text = text |> lowercase |> remove_period |> split_space
@show splitted_text;

splitted_text = SubString{String}["the", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"]


ある単語の前後２単語を周辺語として取ってくる

In [89]:
remove_zero = (range) -> range[range .!= 0]
remove_negative = (range) -> range[range .> 0]
limit_range = (maxlength) -> (range) -> range[range .<= maxlength]
slide_window = (i) -> (range) -> range + i
sample = function (window)
    range = -window:window
    return (maxlength) -> (i) -> range |> remove_zero |> 
    slide_window(i) |> remove_negeative |> limit_range(maxlength)
end

(::#235) (generic function with 1 method)

In [134]:
pos = 1
gen_tsample = (window) -> function(splitted_text) 
    sample_ = sample(window)(length(splitted_text))
    function (pos) 
        sample_indices = sample_(pos)
        sample2 = splitted_text[sample_indices]
        tsamples = []
        for sample in sample2 push!(tsamples, [splitted_text[pos], sample]) end
        return tsamples
    end
end

(::#319) (generic function with 1 method)

In [136]:
gen_tsamples = (window) -> function (splitted_text)
    sample_generator = gen_tsample(window)(splitted_text)
    [sample_generator(pos) for pos in 1:length(splitted_text)] |> flatten
end

(::#329) (generic function with 1 method)

In [139]:
X_train = gen_tsamples(2)(splitted_text)
@show X_train;

X_train = Base.Iterators.Flatten{Array{Array{Any,1},1}}(Array{Any,1}[Any[SubString{String}["the", "quick"], SubString{String}["the", "brown"]], Any[SubString{String}["quick", "the"], SubString{String}["quick", "brown"], SubString{String}["quick", "fox"]], Any[SubString{String}["brown", "the"], SubString{String}["brown", "quick"], SubString{String}["brown", "fox"], SubString{String}["brown", "jumps"]], Any[SubString{String}["fox", "quick"], SubString{String}["fox", "brown"], SubString{String}["fox", "jumps"], SubString{String}["fox", "over"]], Any[SubString{String}["jumps", "brown"], SubString{String}["jumps", "fox"], SubString{String}["jumps", "over"], SubString{String}["jumps", "the"]], Any[SubString{String}["over", "fox"], SubString{String}["over", "jumps"], SubString{String}["over", "the"], SubString{String}["over", "lazy"]], Any[SubString{String}["the", "jumps"], SubString{String}["the", "over"], SubString{String}["the", "lazy"], SubString{String}["the", "dog"]], Any[SubString{Stri

# Model Details

![](http://mccormickml.com/assets/word2vec/skip_gram_net_arch.png)